In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle
import numpy as np

2025-08-16 04:06:21.152222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-16 04:06:21.261787: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-16 04:06:26.017062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import sys
sys.path.append('../../../../')

## load

In [3]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-08-16 04:06:38.267913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test', 'coordinates_train', 'coordinates_validation', 'coordinates_test'])

In [5]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

## debug pairwise transition

In [6]:
from model import build_model

In [7]:
build_model(features_mlp_hidden_sizes=[[256, 256]],
            features_mlp_dropout_rate=0.2,
            output_mlp_hidden_sizes=[[256, 256]], output_mlp_dropout_rate=0.2,
            attention_mlp_hidden_sizes=[[256, 256]], attention_mlp_dropout_rate=0.2,
            activation='relu', input_shape=(10, 9), max_number_of_patches=10, attention_dimension=256,
            gaussian_xrange=(0, 200),
            pairs_channel_dimension=256, num_heads=8, use_pair_bias=False)

initializing projection for skip connection: 11 -> 256
initializing projection for skip connection: None -> 256


TypeError: Exception encountered when calling layer "transformer_encoder_mlp_2" (type TransformerEncoderMLP).

in user code:

    File "/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/models/patch_to_score/models/patch_attention/../../../../models/patch_to_score/models/modules/transformer_encoder_mlp.py", line 44, in call  *
        x = self.dense1(inputs)
    File "/home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib/python3.10/site-packages/keras/engine/input_spec.py", line 213, in assert_input_compatibility
        raise TypeError(f"Inputs to a layer should be tensors. Got: {x}")

    TypeError: Inputs to a layer should be tensors. Got: None


Call arguments received by layer "transformer_encoder_mlp_2" (type TransformerEncoderMLP):
  • inputs=None
  • traini

In [ ]:
from model import create_masked_inputs

features, pairwise_distances = create_masked_inputs(
        input_data, coordinates, size_value, n_patches_hot_encoded_value, max_number_of_patches)

In [ ]:
# px.imshow(pairwise_distances[0][:, :, 0])

In [ ]:
# px.imshow(pairwise_distances._keras_mask[0])

In [ ]:
from models.patch_to_score.models.modules.scannet_layers import GaussianKernel, MaskedDense, initialize_GaussianKernel, initialize_GaussianKernelRandom

In [ ]:
# flat_distances = pairwise_distances.numpy().flatten()
# flat_distances = flat_distances[flat_distances != 0]
# px.histogram(flat_distances[::10])

In [ ]:
N = 20
num_heads = 1
# initial_values = initialize_GaussianKernelRandom([[0, 200]], N, 'diag')
initial_values = [np.array([np.linspace(0, 200, N)]), np.array([[(200 / (N/ 4)) for _ in range(N)]])]
kernel = GaussianKernel(N, initial_values, 'diag')

layer_norm = tf.keras.layers.LayerNormalization()
dense = MaskedDense(num_heads, use_bias=False)

after_kernel = kernel(pairwise_distances)
after_layer_norm = layer_norm(after_kernel)
after_dense = dense(after_layer_norm)

In [ ]:
x = pairwise_distances.numpy().flatten()
ys = [after_kernel[:, :, :, i].numpy().flatten() for i in range(N)]
px.scatter(x=x[::1111], y=[y[::1111] for y in ys])

In [ ]:
# px.imshow(pairwise_distances[0][:, :, 0])

In [ ]:
px.imshow(after_dense[0][:, :, 0])